# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [128]:
# API Key setup
import os

api = !cat ../`ls -a ../ | grep "gemini"`

api = api[0]

os.environ['GOOGLE_API_KEY'] = api
# universal variable, used by langchain

In [7]:
#!pip install python-dotenv
#!pip install openai

In [21]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

# Chat API: Google AI + LangChain

- We'll use LangChain to interact with Google's LLM

- Original course was with ChatOpenAI but due to API restrictions, we'll use google's APIs

In [133]:
#!pip install --upgrade langchain
#!pip install google-generativeai

In [129]:
from langchain.chat_models import ChatOpenAI, ChatGooglePalm

## Model

In [135]:
# Temperature - parameter used to control the randomness of the response

# chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat = ChatGooglePalm(temperature=0.0, model='Bison')

# Created an AI wrapper around Google's model


## Prompt Template

In [156]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
\
text: ```{text}```\
Also, no need of boilerplate text. Just directly give me the answer
"""

template_string

'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```Also, no need of boilerplate text. Just directly give me the answer\n'

In [137]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [138]:
prompt_template.input_variables

['style', 'text']

In [139]:
prompt_template.messages[0].prompt.template

'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```'

- LangChain converted the string template to a more unified version that can be used across different LLMs
- It automatically interpreted the variables that we had provided in the format: between {}


In [162]:
customer_style = '''American English \
in a calm and respectful tone\
'''

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""


In [163]:
# Let's pass in the values to the variable

customer_msg = prompt_template.format_messages(
    style = customer_style,
    text = customer_email
    )

In [164]:
customer_msg[0].content

"Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```"

In [165]:
type(customer_msg)

list

In [166]:
type(customer_msg[0])

langchain_core.messages.human.HumanMessage

In [167]:
customer_resp = chat(customer_msg)

In [168]:
print(customer_resp.content)

Sure, I can help you with that. Here is the text translated into American English in a calm and respectful tone:

"I'm really angry that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, please!"

I hope this is helpful!


In [169]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""



In [171]:
service_msg = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

In [172]:
service_msg[0].content

"Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```"

In [173]:
service_resp = chat(service_msg)
service_resp.content

"Ahoy there, matey! The warranty does not cover cleaning expenses for yer kitchen because it be yer fault that ye misused yer blender by forgettin' to put the lid on before startin' the blender. Tough luck! See ye later!"

## Output Parsers

- We can also define how we would like the LLM output to look like. 
- The output can be formatted

In [174]:
# This is how we expect out output to be:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}


{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [208]:
# Review by the customer
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# Prompt template to tell LLM to get the details and 
# export in JSON
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

Note: I need a direct answer. \
No explation, no unneccesary text \
to explain the output. \
Just pure JSON output \
 \
text: {text}
"""

In [209]:
# Converting prompt string to LangChain's prompt format
prompt_template = ChatPromptTemplate.from_template(review_template)


In [210]:
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\nNote: I need a direct answer. No explation, no unneccesary text to explain the output. Just pure JSON output  text: {text}\n'))])

In [211]:
messages = prompt_template.format_messages(
                            text = customer_review    
                           )

response = chat(messages)

print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["just in time for my wife's anniversary present"]
}
```


- It can be seen that the output is a string
- The below code results in an error because we cannot use it as a json

In [195]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse LLM output string into a python dictionary

- To format the output, we need to create schemas
- let's see that:

In [198]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser


In [202]:
gift_schema = ResponseSchema(name='gift',
                            description="""Was the item purchased as a gift for someone else? \ 
                            Answer True if yes,\
                            False if not or Unknown.""")

delivery_days_schema = ResponseSchema(name='delivery_days',
                                      description="""How many days \
                                      did it take for the product \
                                      to arrive? If this info \
                                      is not found, output -1.""")

price_value_schema = ResponseSchema(name="price_value",
                                    description="""Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.""")

response_schema = [gift_schema, 
                   delivery_days_schema, 
                   price_value_schema]


In [204]:
gift_schema

ResponseSchema(name='gift', description='Was the item purchased as a gift for someone else? \\ \n                            Answer True if yes,                            False if not or Unknown.', type='string')

In [205]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)


In [206]:
format_instructions = output_parser.get_format_instructions()

In [213]:
# It defines the output format
# Similar to how I defined it manually
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as a gift for someone else? \\ \n                            Answer True if yes,                            False if not or Unknown.\n\t"delivery_days": string  // How many days                                       did it take for the product                                       to arrive? If this info                                       is not found, output -1.\n\t"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.\n}\n```'

In [214]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""
# We included the format instructions to the template



In [215]:
prompt = ChatPromptTemplate.from_template(review_template_2)

messages = prompt.format_messages(text=customer_review,
                                  format_instructions=format_instructions
                                 )

In [216]:
messages[0].content

'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "

In [217]:
response = chat(messages)
print(response.content)

```json
{
  "gift": "True",
  "delivery_days": "2",
  "price_value": ["slightly more expensive than the other leaf blowers out there"]
}
```

The following is the reasoning behind the output:

* The item was purchased as a gift for someone else, as evidenced by the sentence "It arrived in two days, just in time for my wife's anniversary present."
* The product arrived in two days.
* The price of the product is not explicitly stated, but it is mentioned that it is "slightly more expensive than the other leaf blowers out there."


In [218]:
output_dict = output_parser.parse(response.content)

In [220]:
# The output doesn't really care about any other text
# Simply scrapes the relevant section of the response
# and parses it into the required format
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ['slightly more expensive than the other leaf blowers out there']}

In [221]:
type(output_dict)

dict

In [222]:
output_dict.get('delivery_days')

'2'